In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('localhost', 27017)

db = client.companies_v1
print(db)


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies_v1')


In [18]:
#CREACIÓN DEL NUEVO DATASET PREPARADO PARA REALIZAR LAS GEOQUERIES.
oficina = db.companies_v1.find({})
dataset1 = pd.DataFrame(oficina)
dataset1.head()

,_id,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices
0,5cd5954ae217c00db4dcb3ac,web,2006,"{'coordinates': [-118.393064, 34.090368], 'typ...",34.090368,-118.393064,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393..."
1,5cd5954ae217c00db4dcb3ad,web,2007,"{'coordinates': [-118.487267, 34.017606], 'typ...",34.017606,-118.487267,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487..."
2,5cd5954ae217c00db4dcb3ae,games_video,2006,"{'coordinates': [-122.409173, 37.788482], 'typ...",37.788482,-122.409173,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409..."
3,5cd5954ae217c00db4dcb3af,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306..."
4,5cd5954ae217c00db4dcb3b0,games_video,2006,"{'coordinates': [-118.254558, 34.051409], 'typ...",34.051409,-118.254558,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254..."


In [3]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 9 columns):
_id                    1498 non-null object
category_code          1498 non-null object
founded_year           1498 non-null int64
geojson                1498 non-null object
latitude               1498 non-null float64
longitude              1498 non-null float64
name                   1498 non-null object
number_of_employees    1498 non-null int64
offices                1498 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 105.4+ KB


In [5]:
#GEOQUERIES.
def empleados_cerca(a, b, dist=600):
    df_temporal1 = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ a , b ]
       },
       "$maxDistance": dist, # In meters
     }
   }
}))
    return sum(df_temporal1.number_of_employees)

In [6]:
dataset1['emp_a_600'] = dataset1.apply(lambda x: empleados_cerca(x['longitude'], x['latitude'], dist=600),axis=1)
dataset1.head()

,_id,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices,emp_a_600
0,5cd5954ae217c00db4dcb3ac,web,2006,"{'coordinates': [-118.393064, 34.090368], 'typ...",34.090368,-118.393064,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393...",18
1,5cd5954ae217c00db4dcb3ad,web,2007,"{'coordinates': [-118.487267, 34.017606], 'typ...",34.017606,-118.487267,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487...",48
2,5cd5954ae217c00db4dcb3ae,games_video,2006,"{'coordinates': [-122.409173, 37.788482], 'typ...",37.788482,-122.409173,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409...",126
3,5cd5954ae217c00db4dcb3af,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306...",101
4,5cd5954ae217c00db4dcb3b0,games_video,2006,"{'coordinates': [-118.254558, 34.051409], 'typ...",34.051409,-118.254558,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254...",70


In [7]:
def empleados_cerca(a, b, dist=1000):
    df_temporal1 = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ a , b ]
       },
       "$maxDistance": dist, # In meters
     }
   }
}))
    return sum(df_temporal1.number_of_employees)

In [8]:
dataset1['emp_a_1000'] = dataset1.apply(lambda x: empleados_cerca(x['longitude'], x['latitude'], dist=1000),axis=1)
dataset1.head()

,_id,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices,emp_a_600,emp_a_1000
0,5cd5954ae217c00db4dcb3ac,web,2006,"{'coordinates': [-118.393064, 34.090368], 'typ...",34.090368,-118.393064,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393...",18,18
1,5cd5954ae217c00db4dcb3ad,web,2007,"{'coordinates': [-118.487267, 34.017606], 'typ...",34.017606,-118.487267,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487...",48,88
2,5cd5954ae217c00db4dcb3ae,games_video,2006,"{'coordinates': [-122.409173, 37.788482], 'typ...",37.788482,-122.409173,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409...",126,866
3,5cd5954ae217c00db4dcb3af,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306...",101,101
4,5cd5954ae217c00db4dcb3b0,games_video,2006,"{'coordinates': [-118.254558, 34.051409], 'typ...",34.051409,-118.254558,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254...",70,75


In [9]:
def empresas_cerca(a, b, dist=600):
    df_temporal1 = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ a , b ]
       },
       "$maxDistance": dist, # In meters
     }
   }
}))
    return len(df_temporal1)

In [10]:
dataset1['empresas_a_600'] = dataset1.apply(lambda x: empresas_cerca(x['longitude'], x['latitude'], dist=600),axis=1)
dataset1.head()

,_id,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices,emp_a_600,emp_a_1000,empresas_a_600
0,5cd5954ae217c00db4dcb3ac,web,2006,"{'coordinates': [-118.393064, 34.090368], 'typ...",34.090368,-118.393064,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393...",18,18,1
1,5cd5954ae217c00db4dcb3ad,web,2007,"{'coordinates': [-118.487267, 34.017606], 'typ...",34.017606,-118.487267,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487...",48,88,2
2,5cd5954ae217c00db4dcb3ae,games_video,2006,"{'coordinates': [-122.409173, 37.788482], 'typ...",37.788482,-122.409173,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409...",126,866,8
3,5cd5954ae217c00db4dcb3af,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306...",101,101,5
4,5cd5954ae217c00db4dcb3b0,games_video,2006,"{'coordinates': [-118.254558, 34.051409], 'typ...",34.051409,-118.254558,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254...",70,75,2


In [11]:
def empresas_cerca(a, b, dist=1000):
    df_temporal1 = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ a , b ]
       },
       "$maxDistance": dist, # In meters
     }
   }
}))
    return len(df_temporal1)

In [12]:
dataset1['empresas_a_1000'] = dataset1.apply(lambda x: empresas_cerca(x['longitude'], x['latitude'], dist=1000),axis=1)
dataset1.head()

,_id,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices,emp_a_600,emp_a_1000,empresas_a_600,empresas_a_1000
0,5cd5954ae217c00db4dcb3ac,web,2006,"{'coordinates': [-118.393064, 34.090368], 'typ...",34.090368,-118.393064,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393...",18,18,1,1
1,5cd5954ae217c00db4dcb3ad,web,2007,"{'coordinates': [-118.487267, 34.017606], 'typ...",34.017606,-118.487267,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487...",48,88,2,3
2,5cd5954ae217c00db4dcb3ae,games_video,2006,"{'coordinates': [-122.409173, 37.788482], 'typ...",37.788482,-122.409173,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409...",126,866,8,28
3,5cd5954ae217c00db4dcb3af,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306...",101,101,5,5
4,5cd5954ae217c00db4dcb3b0,games_video,2006,"{'coordinates': [-118.254558, 34.051409], 'typ...",34.051409,-118.254558,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254...",70,75,2,3


In [13]:
dataset1.drop('_id', axis=1, inplace=True)

In [17]:
dataset1.head(100)

,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices,emp_a_600,emp_a_1000,empresas_a_600,empresas_a_1000,600m
0,web,2006,"{'coordinates': [-118.393064, 34.090368], 'typ...",34.090368,-118.393064,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393...",18,18,1,1,18
1,web,2007,"{'coordinates': [-118.487267, 34.017606], 'typ...",34.017606,-118.487267,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487...",48,88,2,3,48
2,games_video,2006,"{'coordinates': [-122.409173, 37.788482], 'typ...",37.788482,-122.409173,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409...",126,866,8,28,126
3,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306...",101,101,5,5,101
4,games_video,2006,"{'coordinates': [-118.254558, 34.051409], 'typ...",34.051409,-118.254558,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254...",70,75,2,3,70
5,games_video,2007,"{'coordinates': [-73.995625, 40.726155], 'type...",40.726155,-73.995625,Livestream,120,"[{'latitude': 40.726155, 'longitude': -73.9956...",134,224,4,10,134
6,games_video,2007,"{'coordinates': [-74.3372, 40.801358], 'type':...",40.801358,-74.337200,AdaptiveBlue,15,"[{'latitude': 40.801358, 'longitude': -74.3372}]",15,15,1,1,15
7,mobile,2006,"{'coordinates': [-121.932202, 37.465645], 'typ...",37.465645,-121.932202,GrandCentral,3,"[{'latitude': 37.465645, 'longitude': -121.932...",3,3,1,1,3
8,web,2007,"{'coordinates': [-122.397224, 37.762541], 'typ...",37.762541,-122.397224,Pownce,6,"[{'latitude': 37.762541, 'longitude': -122.397...",29,266,4,8,29
9,web,2007,"{'coordinates': [-119.306607, 37.269175], 'typ...",37.269175,-119.306607,SodaHead,25,"[{'latitude': 37.269175, 'longitude': -119.306...",101,101,5,5,101


In [15]:
#DESCARGO EL DATASET Y ME LO LLEVO A TABLEAU PARA VISUALIZARLO Y SACAR CONCLUSIÓN.
dataset1.to_json('dataset1.json', orient='records')

In [ ]:
El lugar ideal para colocar la oficina sería en Mission ST, San Francisco (37.47056, 12224139), 
lugar donde convergen el mayor número de empresas tecnológicas pequeñas, medianas y grandes 
que, a su vez, albergan el una gran concentración de empleados. 

In [16]:
'''
lista=[]
for a,b in zip(dataset1['longitude'], dataset1['latitude']):
    df_temporal = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ a , b ]
       },
       "$maxDistance": 600, # In meters
     }
   }
}))
    suma = sum(df_temporal.number_of_employees)
    lista.append(suma)
print(len(lista))

dataset1['600m']= lista
dataset1.head()
'''


''''
def numero_emp_cerca(geojson, max_distance = 600):
    emp_cerca = pd.DataFrame(list(companies_v1.find(
        {'geojson': { '$near': {'$geometry': geojson, '$maxDistance': max_distance}}})))
    return sum(emp_cerca['number_of_employees'])
''''''
'''
nearLocation = {
    "lng": 37.793568,
    "lat": -122.404621
}

dataset1 = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       },
       "$maxDistance": 600, # In meters
     }
   }
}))
x = sum(dataset1.number_of_employees)
x
'''






1498
